In [1]:
# Standard python modules
import json, time, urllib.parse
import pandas as pd

# The 'requests' module is not a standard Python module. You will need to install this with pip/pip3 if you do not already have it
import requests

## Loading source data

In [83]:
politicians_df = pd.read_csv("source_data/politicians_by_country_AUG.2024.csv")
politicians_df.head()

,name,url,country
0,Majah Ha Adrif,https://en.wikipedia.org/wiki/Majah_Ha_Adrif,Afghanistan
1,Haroon al-Afghani,https://en.wikipedia.org/wiki/Haroon_al-Afghani,Afghanistan
2,Tayyab Agha,https://en.wikipedia.org/wiki/Tayyab_Agha,Afghanistan
3,Khadija Zahra Ahmadi,https://en.wikipedia.org/wiki/Khadija_Zahra_Ah...,Afghanistan
4,Aziza Ahmadyar,https://en.wikipedia.org/wiki/Aziza_Ahmadyar,Afghanistan


In [84]:
population_df = pd.read_csv("source_data/population_by_country_AUG.2024.csv")
population_df.head()

,Geography,Population
0,WORLD,8009.0
1,AFRICA,1453.0
2,NORTHERN AFRICA,256.0
3,Algeria,46.8
4,Egypt,105.2


## Getting Article Quality Predictions
 
Getting a Wikipedia page quality prediction from ORES for each politician’s article page

- Access page info data using the [MediaWiki REST API for the EN Wikipedia](https://www.mediawiki.org/wiki/API:Main_page)
- An example code that how to request summary 'page info' for a single article page is used. This code example was developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/).
- The API documentation, [API:Info](https://www.mediawiki.org/wiki/API:Info), covers additional details that may be helpful when trying to use or understand the code. 

In [4]:
#########
#
#    CONSTANTS
#

# The basic English Wikipedia API endpoint
API_ENWIKIPEDIA_ENDPOINT = "https://en.wikipedia.org/w/api.php"
API_HEADER_AGENT = 'User-Agent'

# We'll assume that there needs to be some throttling for these requests - we should always be nice to a free data resource
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique to the person making the request
# This should include an email - your UW email would be good to put in there
REQUEST_HEADERS = {
    'User-Agent': '<zwang28@uw.edu>, University of Washington, MSDS DATA 512 - AUTUMN 2024'
}

# This is just a list of English Wikipedia article titles that we can use for example requests
ARTICLE_TITLES = [ 'Bison', 'Northern flicker', 'Red squirrel', 'Chinook salmon', 'Horseshoe bat' ]

# This is a string of additional page properties that can be returned see the Info documentation for
# what can be included. If you don't want any this can simply be the empty string
PAGEINFO_EXTENDED_PROPERTIES = "talkid|url|watched|watchers"
#PAGEINFO_EXTENDED_PROPERTIES = ""

# This template lists the basic parameters for making this
PAGEINFO_PARAMS_TEMPLATE = {
    "action": "query",
    "format": "json",
    "titles": "",           # to simplify this should be a single page title at a time
    "prop": "info",
    "inprop": PAGEINFO_EXTENDED_PROPERTIES
}


In [5]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_pageinfo_per_article(article_title = None, 
                                 endpoint_url = API_ENWIKIPEDIA_ENDPOINT, 
                                 request_template = PAGEINFO_PARAMS_TEMPLATE,
                                 headers = REQUEST_HEADERS):
    """
    
    """
    
    # article title can be as a parameter to the call or in the request_template
    if article_title:
        request_template['titles'] = article_title

    if not request_template['titles']:
        raise Exception("Must supply an article title to make a pageinfo request.")

    if API_HEADER_AGENT not in headers:
        raise Exception(f"The header data should include a '{API_HEADER_AGENT}' field that contains your UW email address.")

    if 'uwnetid@uw' in headers[API_HEADER_AGENT]:
        raise Exception(f"Use your UW email address in the '{API_HEADER_AGENT}' field.")

    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or any other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(endpoint_url, headers=headers, params=request_template)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


In [6]:
page_titles = f"{ARTICLE_TITLES[0]}|{ARTICLE_TITLES[2]}|{ARTICLE_TITLES[4]}"
print(f"Getting page info data for: {page_titles}")
request_info = PAGEINFO_PARAMS_TEMPLATE.copy()
request_info['titles'] = page_titles
info = request_pageinfo_per_article(request_template=request_info)
print(json.dumps(info['query']['pages'],indent=4))

Getting page info data for: Bison|Red squirrel|Horseshoe bat
{
    "4583": {
        "pageid": 4583,
        "ns": 0,
        "title": "Bison",
        "contentmodel": "wikitext",
        "pagelanguage": "en",
        "pagelanguagehtmlcode": "en",
        "pagelanguagedir": "ltr",
        "touched": "2024-10-08T07:51:29Z",
        "lastrevid": 1244199338,
        "length": 60757,
        "watchers": 260,
        "talkid": 75239,
        "fullurl": "https://en.wikipedia.org/wiki/Bison",
        "editurl": "https://en.wikipedia.org/w/index.php?title=Bison&action=edit",
        "canonicalurl": "https://en.wikipedia.org/wiki/Bison"
    },
    "531505": {
        "pageid": 531505,
        "ns": 0,
        "title": "Horseshoe bat",
        "contentmodel": "wikitext",
        "pagelanguage": "en",
        "pagelanguagehtmlcode": "en",
        "pagelanguagedir": "ltr",
        "touched": "2024-10-10T19:27:23Z",
        "lastrevid": 1240735439,
        "length": 57103,
        "watchers": 64,
 

### Requesting ORES Scores through LiftWing ML Service API

- Wikimedia Foundation (WMF) is reworking access to their APIs. It is likely in the coming years that all API access will require some kind of authentication, either through a simple key/token or through some version of OAuth. For now this is still a work in progress. You can follow the progress from their [API portal](https://api.wikimedia.org/wiki/Main_Page). 
- Another on-going change is better control over API services in situations where those services require additional computational resources, beyond simply serving the text of a web page (i.e., the text of an article). Services like ORES that require running an ML model over the text of an article page is an example of a compute intensive API service.

- Wikimedia is implementing a new Machine Learning (ML) service infrastructure that they call [LiftWing](https://wikitech.wikimedia.org/wiki/Machine_Learning/LiftWing). Given that ORES already has several ML models that have been well used, ORES is the first set of APIs that are being moved to LiftWing.

- This example illustrates how to generate article quality estimates for article revisions using the LiftWing version of [ORES](https://www.mediawiki.org/wiki/ORES). 
- The [ORES API documentation](https://ores.wikimedia.org) can be accessed from the main ORES page. The [ORES LiftWing documentation](https://wikitech.wikimedia.org/wiki/Machine_Learning/LiftWing/Usage) is very thin ... even thinner than the standard ORES documentation.

- An code example that was developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program is used. This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). Revision 1.0 - August 15, 2023

In [9]:
#########
#
#    CONSTANTS
#

#    The current LiftWing ORES API endpoint and prediction model
#
API_ORES_LIFTWING_ENDPOINT = "https://api.wikimedia.org/service/lw/inference/v1/models/{model_name}:predict"
API_ORES_EN_QUALITY_MODEL = "enwiki-articlequality"

#
#    The throttling rate is a function of the Access token that you are granted when you request the token. The constants
#    come from dissecting the token and getting the rate limits from the granted token. An example of that is below.
#
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = ((60.0*60.0)/5000.0)-API_LATENCY_ASSUMED  # The key authorizes 5000 requests per hour

#    When making automated requests we should include something that is unique to the person making the request
#    This should include an email - your UW email would be good to put in there
#    
#    Because all LiftWing API requests require some form of authentication, you need to provide your access token
#    as part of the header too
#
REQUEST_HEADER_TEMPLATE = {
    'User-Agent': "<{email_address}>, University of Washington, MSDS DATA 512 - AUTUMN 2024",
    'Content-Type': 'application/json',
    'Authorization': "Bearer {access_token}"
}
#
#    This is a template for the parameters that we need to supply in the headers of an API request
#
REQUEST_HEADER_PARAMS_TEMPLATE = {
    'email_address' : "zwang28@uw.edu",         # your email address should go here
    'access_token'  : ""          # the access token you create will need to go here
}

#
#    A dictionary of English Wikipedia article titles (keys) and sample revision IDs that can be used for this ORES scoring example
#
ARTICLE_REVISIONS = { 'Bison':1085687913 , 'Northern flicker':1086582504 , 'Red squirrel':1083787665 , 'Chinook salmon':1085406228 , 'Horseshoe bat':1060601936 }

#
#    This is a template of the data required as a payload when making a scoring request of the ORES model
#
ORES_REQUEST_DATA_TEMPLATE = {
    "lang":        "en",     # required that its english - we're scoring English Wikipedia revisions
    "rev_id":      "",       # this request requires a revision id
    "features":    True
}

#
#    These are used later - defined here so they, at least, have empty values
#
USERNAME = ""
ACCESS_TOKEN = ""
#

In [21]:
#########
##### MAKE SURE TO DELETE BEFORE PUSHING TO GITHUB!!!
USERNAME = "Zy2wang"
ACCESS_TOKEN = "<secret access token>"

In [23]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_ores_score_per_article(article_revid = None, email_address=None, access_token=None,
                                   endpoint_url = API_ORES_LIFTWING_ENDPOINT, 
                                   model_name = API_ORES_EN_QUALITY_MODEL, 
                                   request_data = ORES_REQUEST_DATA_TEMPLATE, 
                                   header_format = REQUEST_HEADER_TEMPLATE, 
                                   header_params = REQUEST_HEADER_PARAMS_TEMPLATE):
    
    #    Make sure we have an article revision id, email and token
    #    This approach prioritizes the parameters passed in when making the call
    if article_revid:
        request_data['rev_id'] = article_revid
    if email_address:
        header_params['email_address'] = email_address
    if access_token:
        header_params['access_token'] = access_token
    
    #   Making a request requires a revision id - an email address - and the access token
    if not request_data['rev_id']:
        raise Exception("Must provide an article revision id (rev_id) to score articles")
    if not header_params['email_address']:
        raise Exception("Must provide an 'email_address' value")
    if not header_params['access_token']:
        raise Exception("Must provide an 'access_token' value")
    
    # Create the request URL with the specified model parameter - default is a article quality score request
    request_url = endpoint_url.format(model_name=model_name)
    
    # Create a compliant request header from the template and the supplied parameters
    headers = dict()
    for key in header_format.keys():
        headers[str(key)] = header_format[key].format(**header_params)
    
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free data
        # source like ORES - or other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        #response = requests.get(request_url, headers=headers)
        response = requests.post(request_url, headers=headers, data=json.dumps(request_data))
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


In [24]:
#   
#
#   Which article - the key for the article dictionary defined above
article_title = "Bison"
#
print(f"Getting LiftWing ORES scores for '{article_title}' with revid: {ARTICLE_REVISIONS[article_title]:d}")
#
#    Make the call, just pass in the article revision ID, email address, and access token
score = request_ores_score_per_article(article_revid=ARTICLE_REVISIONS[article_title],
                                       email_address="dwmc@uw.edu",
                                       access_token=ACCESS_TOKEN)
#
#    Output the result
print(json.dumps(score,indent=4))
#

Getting LiftWing ORES scores for 'Bison' with revid: 1085687913
{
    "enwiki": {
        "models": {
            "articlequality": {
                "version": "0.9.2"
            }
        },
        "scores": {
            "1085687913": {
                "articlequality": {
                    "score": {
                        "prediction": "FA",
                        "probability": {
                            "B": 0.07895665991827401,
                            "C": 0.03728215742560417,
                            "FA": 0.5629436065906797,
                            "GA": 0.30547854835374505,
                            "Start": 0.011061807252218824,
                            "Stub": 0.00427722045947826
                        }
                    }
                }
            }
        }
    }
}


In [42]:
article_no_score = []
articles_no_revid = []
final_data = []

# read each line of politicians_df
for index, row in politicians_df.iterrows():
    article_title = row['name'] # e.g. Majah Ha Adrif
    country = row['country']  
    
    # make a page info request to get the current page revision
    request_info = PAGEINFO_PARAMS_TEMPLATE.copy()
    request_info['titles'] = article_title
    page_info = request_pageinfo_per_article(request_template=request_info)
    
    if page_info is None:
        print(f"No response received for article: {article_title}")
        articles_no_revid.append(article_title)
        continue
        
    pages = page_info.get('query', {}).get('pages', {})
    
    for page_id, page_data in pages.items():
        revision_id = page_data.get('lastrevid')
        
        if revision_id:
            print(f"Got revision ID: {revision_id} for article: {article_title}")
        
        else:
            print(f"No revision ID found for article: {article_title}")
            articles_no_revid.append(article_title)

    # make an ORES request using the page title and current revision id
        print(f"Getting LiftWing ORES scores for '{article_title}' with revid: {revision_id}")

        score = request_ores_score_per_article(article_revid=revision_id,
                                       email_address="zwang28@uw.edu",
                                       access_token=ACCESS_TOKEN)
        if score:
            try:
                prediction = score['enwiki']['scores'][str(revision_id)]['articlequality']['score']['prediction']
                print(f"Prediction for article '{article_title}': {prediction}")
                final_data.append([country, article_title, revision_id, prediction])
            except KeyError:
                print(f"Could not find prediction for article '{article_title}' with revid: {revision_id}")
                article_no_score.append(article_title)
        else:
            article_no_score.append(article_title)
            print(f"No score for article: '{article_title}' with revid: {revision_id}")

final_df = pd.DataFrame(final_data, columns=['country', 'article_title', 'revision_id', 'article_quality'])


Got revision ID: 1233202991 for article: Majah Ha Adrif
Getting LiftWing ORES scores for 'Majah Ha Adrif' with revid: 1233202991
Prediction for article 'Majah Ha Adrif': Start
Got revision ID: 1230459615 for article: Haroon al-Afghani
Getting LiftWing ORES scores for 'Haroon al-Afghani' with revid: 1230459615
Prediction for article 'Haroon al-Afghani': B
Got revision ID: 1225661708 for article: Tayyab Agha
Getting LiftWing ORES scores for 'Tayyab Agha' with revid: 1225661708
Prediction for article 'Tayyab Agha': Start
Got revision ID: 1234741562 for article: Khadija Zahra Ahmadi
Getting LiftWing ORES scores for 'Khadija Zahra Ahmadi' with revid: 1234741562
Prediction for article 'Khadija Zahra Ahmadi': Stub
Got revision ID: 1195651393 for article: Aziza Ahmadyar
Getting LiftWing ORES scores for 'Aziza Ahmadyar' with revid: 1195651393
Prediction for article 'Aziza Ahmadyar': Start
Got revision ID: 1235521766 for article: Muqadasa Ahmadzai
Getting LiftWing ORES scores for 'Muqadasa Ahmad

Got revision ID: 1247762293 for article: Azizullah Lodin
Getting LiftWing ORES scores for 'Azizullah Lodin' with revid: 1247762293
Prediction for article 'Azizullah Lodin': Start
Got revision ID: 1227180106 for article: Razaq Mamoon
Getting LiftWing ORES scores for 'Razaq Mamoon' with revid: 1227180106
Prediction for article 'Razaq Mamoon': Start
Got revision ID: 1234514379 for article: Fazel Ahmed Manawi
Getting LiftWing ORES scores for 'Fazel Ahmed Manawi' with revid: 1234514379
Prediction for article 'Fazel Ahmed Manawi': Start
Got revision ID: 1246567093 for article: Moeen Marastial
Getting LiftWing ORES scores for 'Moeen Marastial' with revid: 1246567093
Prediction for article 'Moeen Marastial': Start
Got revision ID: 1221720658 for article: Ahmad Wali Massoud
Getting LiftWing ORES scores for 'Ahmad Wali Massoud' with revid: 1221720658
Prediction for article 'Ahmad Wali Massoud': Start
Got revision ID: 1227103354 for article: Mohammad Daud Miraki
Getting LiftWing ORES scores for '

Prediction for article 'Azis Tahir Ajdonati': Stub
Got revision ID: 1217284690 for article: Bislim Bajgora
Getting LiftWing ORES scores for 'Bislim Bajgora' with revid: 1217284690
Prediction for article 'Bislim Bajgora': C
Got revision ID: 1244463139 for article: Kastriot Berissa
Getting LiftWing ORES scores for 'Kastriot Berissa' with revid: 1244463139
Prediction for article 'Kastriot Berissa': Start
Got revision ID: 1250227174 for article: Bardhok Biba
Getting LiftWing ORES scores for 'Bardhok Biba' with revid: 1250227174
Prediction for article 'Bardhok Biba': C
Got revision ID: 1250227994 for article: Kostandin Boshnjaku
Getting LiftWing ORES scores for 'Kostandin Boshnjaku' with revid: 1250227994
Prediction for article 'Kostandin Boshnjaku': C
Got revision ID: 1224650124 for article: Spartak Braho
Getting LiftWing ORES scores for 'Spartak Braho' with revid: 1224650124
Prediction for article 'Spartak Braho': Start
Got revision ID: 1189442848 for article: Safet Butka
Getting LiftWing

Got revision ID: 1180949774 for article: Thoma Orologa
Getting LiftWing ORES scores for 'Thoma Orologa' with revid: 1180949774
Prediction for article 'Thoma Orologa': Start
Got revision ID: 1220757655 for article: Bedri Pejani
Getting LiftWing ORES scores for 'Bedri Pejani' with revid: 1220757655
Prediction for article 'Bedri Pejani': C
Got revision ID: 1198167947 for article: Kadri Prishtina
Getting LiftWing ORES scores for 'Kadri Prishtina' with revid: 1198167947
Prediction for article 'Kadri Prishtina': Start
Got revision ID: 1215667416 for article: Ymer Prizreni
Getting LiftWing ORES scores for 'Ymer Prizreni' with revid: 1215667416
Prediction for article 'Ymer Prizreni': Start
Got revision ID: 1229276965 for article: Avni Rustemi
Getting LiftWing ORES scores for 'Avni Rustemi' with revid: 1229276965
Prediction for article 'Avni Rustemi': Start
Got revision ID: 1141930406 for article: Chatin Sarachi
Getting LiftWing ORES scores for 'Chatin Sarachi' with revid: 1141930406
Prediction

Got revision ID: 1230623499 for article: Ibrahim Boughali
Getting LiftWing ORES scores for 'Ibrahim Boughali' with revid: 1230623499
Prediction for article 'Ibrahim Boughali': Stub
Got revision ID: 1186319308 for article: Yahia Boukhari
Getting LiftWing ORES scores for 'Yahia Boukhari' with revid: 1186319308
Prediction for article 'Yahia Boukhari': Stub
Got revision ID: 1224118485 for article: Mohamed Bouslimani
Getting LiftWing ORES scores for 'Mohamed Bouslimani' with revid: 1224118485
Prediction for article 'Mohamed Bouslimani': Stub
Got revision ID: 1242037866 for article: Foued Chehat
Getting LiftWing ORES scores for 'Foued Chehat' with revid: 1242037866
Prediction for article 'Foued Chehat': Stub
Got revision ID: 1248318178 for article: Hamza Al Sid Cheikh
Getting LiftWing ORES scores for 'Hamza Al Sid Cheikh' with revid: 1248318178
Prediction for article 'Hamza Al Sid Cheikh': Stub
Got revision ID: 1242619010 for article: Hocine Cherhabil
Getting LiftWing ORES scores for 'Hocine

...

Prediction for article 'Vladimir Gessen (Venezuelan politician)': Stub
Got revision ID: 1246246427 for article: Jorge Giordani
Getting LiftWing ORES scores for 'Jorge Giordani' with revid: 1246246427
Prediction for article 'Jorge Giordani': C
Got revision ID: 1218574777 for article: César González Martínez
Getting LiftWing ORES scores for 'César González Martínez' with revid: 1218574777
Prediction for article 'César González Martínez': C
Got revision ID: 1248971751 for article: Luis Graterol Caraballo
Getting LiftWing ORES scores for 'Luis Graterol Caraballo' with revid: 1248971751
Prediction for article 'Luis Graterol Caraballo': Stub
Got revision ID: 1242552557 for article: Liborio Guarulla
Getting LiftWing ORES scores for 'Liborio Guarulla' with revid: 1242552557
Prediction for article 'Liborio Guarulla': C
Got revision ID: 1218120878 for article: José Bernabé Gutiérrez
Getting LiftWing ORES scores for 'José Bernabé Gutiérrez' with revid: 1218120878
Prediction for article 'José Bern

Prediction for article 'Đinh Xuân Quảng': B
Got revision ID: 1238520532 for article: Kiem Do
Getting LiftWing ORES scores for 'Kiem Do' with revid: 1238520532
Prediction for article 'Kiem Do': Start
Got revision ID: 1229375855 for article: Đỗ Quang Giai
Getting LiftWing ORES scores for 'Đỗ Quang Giai' with revid: 1229375855
Prediction for article 'Đỗ Quang Giai': C
Got revision ID: 1193882673 for article: Hồ Đắc Điềm
Getting LiftWing ORES scores for 'Hồ Đắc Điềm' with revid: 1193882673
Prediction for article 'Hồ Đắc Điềm': Stub
Got revision ID: 1238255201 for article: Hồ Văn Nhựt
Getting LiftWing ORES scores for 'Hồ Văn Nhựt' with revid: 1238255201
Prediction for article 'Hồ Văn Nhựt': C
Got revision ID: 1241426352 for article: Hoàng Cao Khải
Getting LiftWing ORES scores for 'Hoàng Cao Khải' with revid: 1241426352
Prediction for article 'Hoàng Cao Khải': Stub
Got revision ID: 1249314533 for article: Hoang Van Chi
Getting LiftWing ORES scores for 'Hoang Van Chi' with revid: 1249314533
P

Prediction for article 'Ali Al Bukhaiti': Stub
Got revision ID: 1243942923 for article: Bruce Conde
Getting LiftWing ORES scores for 'Bruce Conde' with revid: 1243942923
Prediction for article 'Bruce Conde': Start
Got revision ID: 1189134411 for article: Abdul Wahab Al-Dailami
Getting LiftWing ORES scores for 'Abdul Wahab Al-Dailami' with revid: 1189134411
Prediction for article 'Abdul Wahab Al-Dailami': Start
Got revision ID: 1106368440 for article: Ali Ahmad Nasser al-Dhahab
Getting LiftWing ORES scores for 'Ali Ahmad Nasser al-Dhahab' with revid: 1106368440
Prediction for article 'Ali Ahmad Nasser al-Dhahab': Stub
Got revision ID: 1248072502 for article: Badreddin al-Houthi
Getting LiftWing ORES scores for 'Badreddin al-Houthi' with revid: 1248072502
Prediction for article 'Badreddin al-Houthi': Stub
Got revision ID: 1144463479 for article: Ibrahim Bin Ali Al Wazeer
Getting LiftWing ORES scores for 'Ibrahim Bin Ali Al Wazeer' with revid: 1144463479
Prediction for article 'Ibrahim Bi

Prediction for article 'Tonderai Kasu': C
Got revision ID: 1246239559 for article: Wilson Khumbula
Getting LiftWing ORES scores for 'Wilson Khumbula' with revid: 1246239559
Prediction for article 'Wilson Khumbula': C
Got revision ID: 1138179170 for article: Lovemore Madhuku
Getting LiftWing ORES scores for 'Lovemore Madhuku' with revid: 1138179170
Prediction for article 'Lovemore Madhuku': C
Got revision ID: 1246239576 for article: Sarah Mahoka
Getting LiftWing ORES scores for 'Sarah Mahoka' with revid: 1246239576
Prediction for article 'Sarah Mahoka': Stub
Got revision ID: 1092395186 for article: Alfred Makwarimba
Getting LiftWing ORES scores for 'Alfred Makwarimba' with revid: 1092395186
Prediction for article 'Alfred Makwarimba': Stub
Got revision ID: 1064532257 for article: Washington Malianga
Getting LiftWing ORES scores for 'Washington Malianga' with revid: 1064532257
Prediction for article 'Washington Malianga': Stub
Got revision ID: 1246001588 for article: Elliot Manyika
Gettin

Prediction for article 'Denis Walker': C
score_error_rate: 0.1816911250873515


In [47]:
# the ratio of the number of articles we were not able to get a score / the total number of articles
score_error_rate = (len(article_no_score) / len(politicians_df)) 
print(f"score_error_rate: {score_error_rate}")


score_error_rate: 0.001816911250873515


In [59]:
final_df.to_csv('output/Article_Quality_Predictions_data.csv', index=False)

In [86]:
final_df

,country,article_title,revision_id,article_quality
0,Afghanistan,Majah Ha Adrif,1233202991,Start
1,Afghanistan,Haroon al-Afghani,1230459615,B
2,Afghanistan,Tayyab Agha,1225661708,Start
3,Afghanistan,Khadija Zahra Ahmadi,1234741562,Stub
4,Afghanistan,Aziza Ahmadyar,1195651393,Start
...,...,...,...,...
7137,Zimbabwe,Josiah Tongogara,1203429435,C
7138,Zimbabwe,Langton Towungana,1246280093,Stub
7139,Zimbabwe,Sengezo Tshabangu,1228478288,Start
7140,Zimbabwe,Herbert Ushewokunze,959111842,Stub


### Combining the Datasets

merge the wikipedia data and population data together

In [87]:
final_df # wikipedia data

,country,article_title,revision_id,article_quality
0,Afghanistan,Majah Ha Adrif,1233202991,Start
1,Afghanistan,Haroon al-Afghani,1230459615,B
2,Afghanistan,Tayyab Agha,1225661708,Start
3,Afghanistan,Khadija Zahra Ahmadi,1234741562,Stub
4,Afghanistan,Aziza Ahmadyar,1195651393,Start
...,...,...,...,...
7137,Zimbabwe,Josiah Tongogara,1203429435,C
7138,Zimbabwe,Langton Towungana,1246280093,Stub
7139,Zimbabwe,Sengezo Tshabangu,1228478288,Start
7140,Zimbabwe,Herbert Ushewokunze,959111842,Stub


In [88]:
population_df # population data

,Geography,Population
0,WORLD,8009.0
1,AFRICA,1453.0
2,NORTHERN AFRICA,256.0
3,Algeria,46.8
4,Egypt,105.2
...,...,...
228,Samoa,0.2
229,Solomon Islands,0.8
230,Tonga,0.1
231,Tuvalu,0.0


In [95]:
regions_hierarchy = {}

# read each line of population_df
for index, row in population_df.iterrows():
    geography = row['Geography']  # Geography field in population_df
    population = row['Population']  
    
    # ALL CAPS values are region-level rows
    if geography.isupper():  
        current_region = geography  
        regions_hierarchy[current_region] = {'population': population, 'region': current_region}
    else:
        # Otherwise, it's a country-level row, map country to current region
        regions_hierarchy[geography] = {'population': population, 'region': current_region}

# update final_df with population and region from regions_hierarchy
countries_no_match = []  

for index, row in final_df.iterrows():
    country = row['country']
    
    if country in regions_hierarchy:
        # if country found in regions_hierarchy, update population and region
        final_df.at[index, 'population'] = regions_hierarchy[country]['population']
        final_df.at[index, 'region'] = regions_hierarchy[country]['region']
    else:
        countries_no_match.append(country)

# save countries with no match into 'wp_countries-no_match.txt'
with open('output/wp_countries-no_match.txt', 'w') as f:
    for country in set(countries_no_match): 
        f.write(country + "\n")

# save the combined dataset to a CSV file (wp_politicians_by_country.csv)
final_df.to_csv('output/wp_politicians_by_country.csv', index=False, columns=['country', 'region', 'population', 'article_title', 'revision_id', 'article_quality'])


In [96]:
final_df

,country,article_title,revision_id,article_quality,population,region
0,Afghanistan,Majah Ha Adrif,1233202991,Start,42.4,SOUTH ASIA
1,Afghanistan,Haroon al-Afghani,1230459615,B,42.4,SOUTH ASIA
2,Afghanistan,Tayyab Agha,1225661708,Start,42.4,SOUTH ASIA
3,Afghanistan,Khadija Zahra Ahmadi,1234741562,Stub,42.4,SOUTH ASIA
4,Afghanistan,Aziza Ahmadyar,1195651393,Start,42.4,SOUTH ASIA
...,...,...,...,...,...,...
7137,Zimbabwe,Josiah Tongogara,1203429435,C,16.7,EASTERN AFRICA
7138,Zimbabwe,Langton Towungana,1246280093,Stub,16.7,EASTERN AFRICA
7139,Zimbabwe,Sengezo Tshabangu,1228478288,Start,16.7,EASTERN AFRICA
7140,Zimbabwe,Herbert Ushewokunze,959111842,Stub,16.7,EASTERN AFRICA


### Analysis

- This analysis calculates total-articles-per-capita (a ratio representing the number of articles per person)  and high-quality-articles-per-capita (a ratio representing the number of high quality articles per person) on a country-by-country and regional basis

In [111]:
# considers "high quality" articles to be articles that ORES predicted would be 
# in either the "FA" (featured article) or "GA" (good article) classes.

high_quality_articles_df = final_df[final_df['article_quality'].isin(['FA', 'GA'])]


In [115]:
# group by country to calculate the number of articles and high-quality articles

country_df = final_df.groupby('country').agg(
    total_articles=('article_title', 'count'),  # total number of articles
    population=('population', 'first') 
).reset_index()

country_df.head()

,country,total_articles,population
0,Afghanistan,85,42.4
1,Albania,70,2.7
2,Algeria,71,46.8
3,Angola,57,36.7
4,Antigua and Barbuda,33,0.1


In [113]:
high_quality_df = high_quality_articles_df.groupby('country').agg(
    high_quality_articles=('article_title', 'count')  # total number of high-quality articles
).reset_index()

high_quality_df.head()

,country,high_quality_articles
0,Afghanistan,3
1,Albania,7
2,Algeria,1
3,Angola,2
4,Argentina,1


In [117]:
# merge country_df with high_quality_df
country_df_merged = pd.merge(country_df, high_quality_df, on='country', how='left')


country_df_merged.fillna(0, inplace=True)  # Replace NaN with 0


# calculate total_articles_per_capita and high_quality_articles_per_capita

country_df_merged['total_articles_per_capita'] = country_df_merged['total_articles'] / (country_df_merged['population'])
country_df_merged['high_quality_articles_per_capita'] = country_df_merged['high_quality_articles'] / (country_df_merged['population'])
country_df_merged


,country,total_articles,population,high_quality_articles,total_articles_per_capita,high_quality_articles_per_capita
0,Afghanistan,85,42.4,3.0,2.004717,0.070755
1,Albania,70,2.7,7.0,25.925926,2.592593
2,Algeria,71,46.8,1.0,1.517094,0.021368
3,Angola,57,36.7,2.0,1.553134,0.054496
4,Antigua and Barbuda,33,0.1,0.0,330.000000,0.000000
...,...,...,...,...,...,...
164,Venezuela,56,28.8,1.0,1.944444,0.034722
165,Vietnam,36,98.9,2.0,0.364004,0.020222
166,Yemen,32,34.4,0.0,0.930233,0.000000
167,Zambia,3,20.2,0.0,0.148515,0.000000


Do the same for regional level

In [133]:
final_df.head()

,country,article_title,revision_id,article_quality,population,region
0,Afghanistan,Majah Ha Adrif,1233202991,Start,42.4,SOUTH ASIA
1,Afghanistan,Haroon al-Afghani,1230459615,B,42.4,SOUTH ASIA
2,Afghanistan,Tayyab Agha,1225661708,Start,42.4,SOUTH ASIA
3,Afghanistan,Khadija Zahra Ahmadi,1234741562,Stub,42.4,SOUTH ASIA
4,Afghanistan,Aziza Ahmadyar,1195651393,Start,42.4,SOUTH ASIA


In [132]:
# group by region to calculate the number of articles and high-quality articles

region_df = final_df.groupby('region').agg(
    total_articles=('article_title', 'count'),
).reset_index()

high_quality_df_region = high_quality_articles_df.groupby('region').agg(
    high_quality_articles=('article_title', 'count')  # total number of high-quality articles
).reset_index()

# merge region_df with high_quality_df_region
region_df_merged = pd.merge(region_df, high_quality_df_region, on='region', how='left')


# add population column from population_df
region_df_merged = pd.merge(region_df_merged, population_df, left_on='region', right_on='Geography', how='left')
region_df_merged.rename(columns={'Population': 'population'}, inplace=True)
region_df_merged.drop(columns=['Geography'], inplace=True)

# calculate total_articles_per_capita and high_quality_articles_per_capita
region_df_merged['total_articles_per_capita'] = region_df_merged['total_articles'] / (region_df_merged['population'])
region_df_merged['high_quality_articles_per_capita'] = region_df_merged['high_quality_articles'] / (region_df_merged['population'])
region_df_merged



,region,total_articles,high_quality_articles,population,total_articles_per_capita,high_quality_articles_per_capita
0,CARIBBEAN,217,8,44.0,4.931818,0.181818
1,CENTRAL AMERICA,188,10,182.0,1.032967,0.054945
2,CENTRAL ASIA,106,5,80.0,1.325000,0.062500
3,EAST ASIA,152,3,1648.0,0.092233,0.001820
4,EASTERN AFRICA,663,17,483.0,1.372671,0.035197
5,EASTERN EUROPE,709,38,285.0,2.487719,0.133333
6,MIDDLE AFRICA,229,8,202.0,1.133663,0.039604
7,NORTHERN AFRICA,302,17,256.0,1.179688,0.066406
8,NORTHERN EUROPE,191,9,108.0,1.768519,0.083333
9,OCEANIA,72,1,45.0,1.600000,0.022222


**country_df_merged**:
- contains total articles and high-quality articles for each country

**region_df_merged**:
- contains total articles and high-quality articles for each region



### Results


Results from this analysis is produced in the form of 6 data tables. 



1. **Top 10 countries by coverage**: The 10 countries with the highest total articles per capita (in descending order) .




In [167]:
# filter out countries that are in countries_no_match list becuase they
# don't have population value
filtered_country_df = country_df_merged[~country_df_merged['country'].isin(countries_no_match)]


top_10_countries_by_coverage = filtered_country_df.sort_values(by='total_articles_per_capita', 
                                                      ascending=False).head(10)
top_10_countries_by_coverage[['country']].reset_index(drop=True)

,country
0,Monaco
1,Tuvalu
2,Antigua and Barbuda
3,Federated States of Micronesia
4,Marshall Islands
5,Tonga
6,Barbados
7,Montenegro
8,Seychelles
9,Maldives


2. **Bottom 10 countries by coverage**: The 10 countries with the lowest total articles per capita (in ascending order) .



In [168]:

bottom_10_countries_by_coverage = filtered_country_df.sort_values(by='total_articles_per_capita', 
                                                      ascending=True).head(10)
bottom_10_countries_by_coverage[['country']].reset_index(drop=True)

,country
0,China
1,Ghana
2,India
3,Saudi Arabia
4,Zambia
5,Norway
6,Israel
7,Egypt
8,Cote d'Ivoire
9,Ethiopia


3. **Top 10 countries by high quality**: The 10 countries with the highest high quality articles per capita (in descending order) .


In [156]:
top_10_countries_by_high_quality = filtered_country_df.sort_values(
    by='high_quality_articles_per_capita', ascending=False).head(10)

top_10_countries_by_high_quality[['country']].reset_index(drop=True)



,country
0,Montenegro
1,Luxembourg
2,Albania
3,Kosovo
4,Maldives
5,Lithuania
6,Croatia
7,Guyana
8,Palestinian Territory
9,Slovenia


4. **Bottom 10 countries by high quality**: The 10 countries with the lowest high quality articles per capita (in ascending order).



In [160]:

bottom_10_countries_by_high_quality = filtered_country_df.sort_values(
    by='high_quality_articles_per_capita', ascending=True).head(10)

bottom_10_countries_by_high_quality[['country']].reset_index(drop=True)



,country
0,Zimbabwe
1,Congo
2,Kuwait
3,St. Lucia
4,Cote d'Ivoire
5,St. Kitts and Nevis
6,Solomon Islands
7,Cyprus
8,Singapore
9,Djibouti


5. **Geographic regions by total coverage**: A rank ordered list of geographic regions (in descending order) by total articles per capita.


In [164]:
geographic_regions_by_total_coverage = region_df_merged.sort_values(
    by='total_articles_per_capita', ascending=False
).reset_index(drop=True)
geographic_regions_by_total_coverage[['region']].reset_index(drop=True)

,region
0,SOUTHERN EUROPE
1,CARIBBEAN
2,WESTERN EUROPE
3,EASTERN EUROPE
4,WESTERN ASIA
5,NORTHERN EUROPE
6,SOUTHERN AFRICA
7,OCEANIA
8,EASTERN AFRICA
9,SOUTH AMERICA



6. **Geographic regions by high quality coverage**: Rank ordered list of geographic regions (in descending order) by high quality articles per capita.

In [166]:
geographic_regions_by_high_quality = region_df_merged.sort_values(
    by='high_quality_articles_per_capita', ascending=False
).reset_index(drop=True)
geographic_regions_by_high_quality[['region']].reset_index(drop=True)

,region
0,SOUTHERN EUROPE
1,CARIBBEAN
2,EASTERN EUROPE
3,SOUTHERN AFRICA
4,WESTERN EUROPE
5,WESTERN ASIA
6,NORTHERN EUROPE
7,NORTHERN AFRICA
8,CENTRAL ASIA
9,CENTRAL AMERICA
